In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter

# SAGEMAKER SESSIONN
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# S3 BUCKET
s3_bucket = 'your-s3-bucket'
s3_prefix = 'deep-learning-model/'

# Upload the raw SENSOR data to S3
train_data_uri = sagemaker_session.upload_data(path='data/train', bucket=s3_bucket, key_prefix=s3_prefix + 'train')
validation_data_uri = sagemaker_session.upload_data(path='data/val', bucket=s3_bucket, key_prefix=s3_prefix + 'validation')

# PYTHON Script to train
entry_point_script = 'train.py'  # This will be the Python script containing your deep learning model code

#  TensorFlow Estimator
tf_estimator = TensorFlow(
    entry_point=entry_point_script,
    role=role,
    framework_version='2.5',
    py_version='py37',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    hyperparameters={
        'epochs': 50,
        'batch_size': 64,
        'learning_rate': 0.001,
    },
    script_mode=True,
    input_mode='File',
    output_path=f's3://{s3_bucket}/{s3_prefix}/output',
    base_job_name='deep-learning-raw-sensor'
)

# Define Hyperparameter ranges for tuning
hyperparameter_ranges = {
    'learning_rate': ContinuousParameter(0.0001, 0.1),
    'batch_size': IntegerParameter(32, 128),
    'dropout_rate': ContinuousParameter(0.2, 0.5),
    'num_layers': IntegerParameter(1, 5),
}

objective_metric_name = 'val_accuracy'
objective_type = 'Maximize'  # Could also be 'Minimize' for metrics like loss

# Set up the Tuner
tuner = HyperparameterTuner(
    estimator=tf_estimator,
    objective_metric_name=objective_metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=20,  # Max number of HPO jobs
    max_parallel_jobs=3  # How many can run in parallel
)
input_data = {
    'train': train_data_uri,
    'validation': validation_data_uri
}

# Start hyperparameter tuning job
tuner.fit(inputs=input_data)

# After tuning completes, you can deploy the best model automatically
best_estimator = tuner.best_estimator()
best_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='sensor-data-model-endpoint'
)


ModuleNotFoundError: No module named 'sagemaker'

In [2]:
!pip install sagemaker

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 23.7 MB/s eta 0:00:01
   ----------------------------------- ---- 1.4/1.6 MB 21.6 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 19.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.1 kB ? eta -:--:--
   ---------------------------------------- 139.1/139.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/385.0 kB ? eta -:--:--
   --------------------------------------- 385.0/385.0 kB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/147.8 kB ? eta -:--:--
   ---------------------------------------- 147.8/147.8 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
   ---------------------------------------- 82.1/82.1 kB ? eta 0:00:00
   -------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires qtconsole<5.5.0,>=5.4.2, but you have qtconsole 5.5.2 which is incompatible.
voila 0.5.6 requires nbclient<0.8,>=0.4.0, but you have nbclient 0.10.0 which is incompatible.
